# Project Title
### Data Engineering Capstone Project

#### Project Summary
--describe your project at a high level--

The project follows the follow steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

In [3]:
# Do all imports and installs here
import pandas as pd
import configparser
from datetime import datetime
import os
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.functions import from_unixtime, to_timestamp
from pyspark.sql.functions import col, hour, dayofmonth, dayofweek, month, year, weekofyear
from pyspark.sql.types import StringType
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import monotonically_increasing_id

import utility

from etl import (
    get_spark_session,
    get_immigration_data,
    get_us_demographics_data,
    get_i94_countries,
    get_i94_ports,
    get_i94_states,
    get_i94_travel_modes,
    get_i94_visas,
    cln_immigration_data,
    clean_us_demographics_data,
    cln_ports_data,
    cln_countries_data,
    cln_states_data,
    create_imm_table
    create_demo_table
    logger
)



os.environ['AWS_ACCESS_KEY_ID'] = config['AWS']['AWS_ACCESS_KEY_ID']
os.environ['AWS_SECRET_ACCESS_KEY'] = config['AWS']['AWS_SECRET_ACCESS_KEY']




### Step 1: Scope the Project and Gather Data

#### Scope 
Explain what you plan to do in the project in more detail. What data do you use? What is your end solution look like? What tools did you use? etc>

#### Describe and Gather Data 
Describe the data sets you're using. Where did it come from? What type of information is included? 

In [5]:
# Read in the data here
config = configparser.ConfigParser()
config.read('config.cfg')


us_demographics_df = get_us_demographics_data(spark)
countries_df = get_i94_countries(spark)
ports_df = get_i94_ports(spark)
states_df = get_i94_states(spark)
travel_modes_df = get_i94_travel_modes(spark)
visa_categories_df = get_i94_visas(spark)



I94_DATA_FILE_PATH = config['DATA']['I94_DATA_FILE_PATH']
print(I94_DATA_FILE_PATH)
# df = pd.read_sas(I94_DATA_FILE_PATH, format='sas7bdat', encoding="ISO-8859-1")
# Read local parquet dataset in `df`
I94_LOCAL_DATA_DIR = config['DATA']['I94_LOCAL_DATA_DIR']
print("I94_LOCAL_DATA_DIR: ", I94_LOCAL_DATA_DIR)


In [6]:
df.head(n=10)

,cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,depdate,...,entdepu,matflag,biryear,dtaddto,gender,insnum,airline,admnum,fltno,visatype
0,6.0,2016.0,4.0,692.0,692.0,XXX,20573.0,NaN,NaN,NaN,...,U,NaN,1979.0,10282016,NaN,NaN,NaN,1.897628e+09,NaN,B2
1,7.0,2016.0,4.0,254.0,276.0,ATL,20551.0,1.0,AL,NaN,...,Y,NaN,1991.0,D/S,M,NaN,NaN,3.736796e+09,00296,F1
2,15.0,2016.0,4.0,101.0,101.0,WAS,20545.0,1.0,MI,20691.0,...,NaN,M,1961.0,09302016,M,NaN,OS,6.666432e+08,93,B2
3,16.0,2016.0,4.0,101.0,101.0,NYC,20545.0,1.0,MA,20567.0,...,NaN,M,1988.0,09302016,NaN,NaN,AA,9.246846e+10,00199,B2
4,17.0,2016.0,4.0,101.0,101.0,NYC,20545.0,1.0,MA,20567.0,...,NaN,M,2012.0,09302016,NaN,NaN,AA,9.246846e+10,00199,B2


In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.\
config("spark.jars.repositories", "https://repos.spark-packages.org/").\
config("spark.jars.packages", "saurfang:spark-sas7bdat:2.0.0-s_2.11").\
enableHiveSupport().getOrCreate()

df_spark = spark.read.format('com.github.saurfang.sas.spark').load('../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat')

print("Sparksession")

In [11]:
#write to parquet
df_spark.write.parquet("sas_data")
df_spark=spark.read.parquet("sas_data")

### Step 2: Explore and Assess the Data
#### Explore the Data 
Identify data quality issues, like missing values, duplicate data, etc.

#### Cleaning Steps
Document steps necessary to clean the data

In [ ]:
# Performing cleaning tasks here

cln_immigration_df = clean_immigration_data(immigration_df)
cln_us_demographics_df = clean_us_demographics_data(us_demographics_df)
cln_ports_df = clean_ports_data(ports_df)
cln_countries_df = clean_countries_data(countries_df)
cln_states_df = clean_states_data(states_df)

cln_us_demographics_df.createOrReplaceTempView("cln_us_demographics_df")
Demo_table.createOrReplaceTempView("dim_city_demographics")
ports_df.createOrReplaceTempView("dim_ports")
countries_df.createOrReplaceTempView("dim_country")
states_df.createOrReplaceTempView("dim_us_state")
visa_categories_df.createOrReplaceTempView("dim_visa_category")
travel_modes_df.createOrReplaceTempView("dim_travel_mode")



cln_immigration_df.printSchema()
cln_immigration_df.select("state_code").distinct().count()

cln_ports_df.printSchema()
cln_ports_df.select("state_code").distinct().count()

cln_states_df.printSchema()
cln_states_df.select("state_code").distinct().count()

cln_countries_df.printSchema()
cln_countries_df.select("state_code").distinct().count()


spark.sql(""
         select count (distinct city)
         from cln_us_demographics_df
         "").show()



### Step 3: Define the Data Model
#### 3.1 Conceptual Data Model
Map out the conceptual data model and explain why you chose that model

#### 3.2 Mapping Out Data Pipelines
List the steps necessary to pipeline the data into the chosen data model

### Step 4: Run Pipelines to Model the Data 
#### 4.1 Create the data model
Build the data pipelines to create the data model.

In [ ]:
# Write code here
# Creating Imm table 
imm_table = create_imm_table (spark, cln_immigration_df, cln_countries_df, cln_states_df, cln_ports_df, 
                              visa_categories_df, travel_modes_df) 

# Creating Demographic table
Demo_table = create_demo_table (spark, cln_us_demographics_df, cln_ports_df)


#### 4.2 Data Quality Checks
Explain the data quality checks you'll perform to ensure the pipeline ran as expected. These could include:
 * Integrity constraints on the relational database (e.g., unique key, data type, etc.)
 * Unit tests for the scripts to ensure they are doing the right thing
 * Source/Count checks to ensure completeness
 
Run Quality Checks

In [ ]:
# Perform quality checks here

def quality_check(df, description):
    '''
    Input: Spark dataframe, description of Spark datafram
    
    Output: Print outcome of data quality check
    
    '''
    
    result = df.count()
    
    if result == 0:
        print("Data quality check failed for {imm_table} with zero records".format(description))
    else:
        print("Data quality check passed for {imm_table} with {results:,} records".format(description, result))
    return 0




# Perform data quality check
quality_check(imm_table, "immigration table")
quality_check(demo_table, "temperature table")





spark.sql(""
         select count (*)
         from imm_table
         "").show()


spark.sql(""
         select count (*)
         from Demo_table
         "").show()




#### 4.3 Data dictionary 
Create a data dictionary for your data model. For each field, provide a brief description of what the data is and where it came from. You can include the data dictionary in the notebook or in a separate file.

#### Step 5: Complete Project Write Up
* Clearly state the rationale for the choice of tools and technologies for the project.
* Propose how often the data should be updated and why.
* Write a description of how you would approach the problem differently under the following scenarios:
 * The data was increased by 100x.
 * The data populates a dashboard that must be updated on a daily basis by 7am every day.
 * The database needed to be accessed by 100+ people.

In [ ]:
# Which city was most visited in a specific month?

spark.sql("""
    SELECT
        a.port_code,
        a.immigrant_visits,
        b.city,
        b.state_code,
        b.total_population
    FROM
        (SELECT 
            cit.port_code AS port_code, 
            COUNT(*) AS immigrant_visits
        FROM create_imm_table cit 
        WHERE cit.entry_month = 4
        GROUP BY cit.port_code
        ORDER BY immigrant_visits DESC
        LIMIT 10
        ) AS a 
    JOIN Demo_table b
        ON a.port_code = b.port_code
    ORDER BY b.immigrant_visits DESC
""").show()
# Showing top 10 visited cities here

In [ ]:
#From which country (or countries) travelers originate? Top countries of origin.
spark.sql("""
    SELECT *
    FROM
        (SELECT 
            fi.origin_country_code AS origin_country_code, 
            COUNT(*) AS country_visitors
        FROM fact_immigrations cit 
        GROUP BY cit.origin_country_code
        ORDER BY country_visitors DESC
        LIMIT 10
        ) AS a
    JOIN dim_country b
        ON a.origin_country_code = b.country_code
    ORDER BY country_visitors DESC
""").show()